# Departure by plane model


## Importing
Importing libraries. Setting `%matplotlib inline` to plott the graph inside the notebook

In [21]:
%matplotlib inline

import pandas as pd
import geopandas
from shapely.geometry import Point, Polygon
import fiona

Importing CSV

In [22]:
# Import Points
df_points = pd.read_csv('data/20180521_sampled.csv')

#Import Plane list
df_planes = pd.read_csv('data/swiss_airplanes.csv')

## Join points and airplanes

In [24]:
# set index
df_planes = df_planes.set_index('icao24')

# Join
df_joined = df_points.join(df_planes, on='icao24')

In [27]:
# Only take swiss planes
df_swiss = df_joined[df_joined.status == 'active']

In [17]:


Hier weitermachen


df_points.head()

,timestamp,icao24,latitude,longitude,ground_speed,track,vertical_rate,callsign,onground,alert,spi,squawk,baro_altitude,altitude,last_position,hour,geometry
0,2018-05-21 00:00:02,4b1801,47.441299,8.561869,NaN,NaN,NaN,SWR112W,True,False,False,3072.0,NaN,NaN,2018-05-20 23:43:24.733000,1526853600,POINT (8.561868667602539 47.44129943847656)
1,2018-05-21 00:00:03,4ca640,47.131580,8.040020,450.404984,322.850241,0.0,EIN417,False,False,False,7532.0,35950.0,36325.0,2018-05-21 00:00:02.350000,1526853600,POINT (8.040020282451923 47.1315804174391)
2,2018-05-21 00:00:04,4ca640,47.131580,8.040020,450.404984,322.850241,0.0,EIN417,False,False,False,7532.0,35950.0,36325.0,2018-05-21 00:00:02.350000,1526853600,POINT (8.040020282451923 47.1315804174391)
3,2018-05-21 00:00:05,4b1801,47.441299,8.561869,NaN,NaN,NaN,SWR112W,True,False,False,3072.0,NaN,NaN,2018-05-20 23:43:24.733000,1526853600,POINT (8.561868667602539 47.44129943847656)
4,2018-05-21 00:00:06,3c09f4,47.103975,7.796208,465.690482,35.737912,0.0,GMI54HM,False,False,False,2315.0,38050.0,38325.0,2018-05-21 00:00:05.989000,1526853600,POINT (7.796208308293267 47.10397494041313)


In [7]:
geometry = [Point(xy) for xy in zip(df.latitude, df.longitude)]
geo_df = geopandas.GeoDataFrame(df, crs={'init': 'epsg:4326'}, geometry=geometry)

In [ ]:
# Airport Zurich
coords = [( 47.4863, 8.4994 ), ( 47.4283, 8.4994 ), ( 47.4283, 8.619 ), ( 47.4863, 8.619 ), ( 47.4863, 8.4994 )]
zurich_airport = Polygon(coords)

In [ ]:
df_zurich = geo_df[geo_df.geometry.within(zurich_airport)]

In [ ]:
df_zurich.plot(figsize=(12, 12),markersize=1, alpha=0.4)

## Terminal
Zoom to a terminal - just for fun

Again, create a polygone

In [ ]:
# Terminal
coords = [( 47.4636, 8.5482 ), ( 47.4582, 8.5482 ), ( 47.4582, 8.5625 ), ( 47.4636, 8.5625 ), ( 47.4636, 8.5482 )]
airport_terminal = Polygon(coords)

Just select dots on the ground.

In [ ]:
geo_onground = geo_df[geo_df['onground'] == True]

Select points inside polygon and plott

In [ ]:
df_terminal = geo_onground[geo_onground.geometry.within(airport_terminal)]
df_terminal.plot(figsize=(12, 12), markersize=1)